In [1]:
#동적 생성되는 data 크롤링
#Selenium 모듈 사용
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import selenium
from selenium import webdriver


### 관련 메서드
- get(url) : url에 문서를 요청하고 응답을 받는 메서드
    
#### 페이지 단일 element에 접근하는 메서드
- find_element_by_name('html_name')
- find_element_by_id('html_id')
- find_element_by_xpath('패스경로') : 스크립트 실행 경로를 찾아 접근
    
    
#### 페이지의 여러 element에 접근하는 메서드
- find_element_by_class_selector('html slelctor')
- find_element_by_class_name('html_class')
- find_element_by_tag_name('htm tag')

#### Webdriver 객체 생성
- driver = webdriver.Chrome(드라이버경로)

#### 접근한 페이지 source 추출
- html=driver.page_source


In [3]:
# 크롬 드라이버를 통한 네이버 접속

driver = webdriver.Chrome("./driver/chromedriver")
# driver.get("https://news.naver.com//main/read.naver?mode=LSD&mid=shm&sid1=102&oid=001&aid=0012587876")



SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 92
Current browser version is 94.0.4606.81 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


In [ ]:
driver.page_source # 전체 페이지 소스 로딩

In [21]:

like_list = driver.find_elements_by_css_selector("span.u_likeit_text._count.num")
like_num = [like.text for like in like_list]
like_num

['820']

In [22]:
#articleTitleCommentCount > span.lo_txt


In [25]:
rep_list = driver.find_elements_by_css_selector("span.lo_txt")
rep_num = [rep.text for rep in rep_list]
rep_num

['476']

In [43]:
# 전달된 뉴스 url

def get_news_sub_info(url) :
    
    #1.전달된 url에 접속 후 응답객체 반환
    headers ={"User-Agent" : 
          "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
    result = requests.get(url,headers=headers)
    
    #2.bs4 객체 생성
    bs_obj=BeautifulSoup(result.content,"html.parser")
    
    #3.필요 데이터 추출
    # 기사 제목 추출(태그객체)
    title =bs_obj.find(id='articleTitle')
    # 기사 입력시간 추출(태그객체)
    date_time = bs_obj.find("span",{"class":"t11"})
    
    #4. webdriver 이용해서 좋아요 수를 반환
#     driver = webdriver.Chrome("./driver/chromedriver")
    time.sleep(2)
    driver.get(url)
    driver.page_source
    
    # 좋아요 수
    like_list = driver.find_elements_by_css_selector("span.u_likeit_text._count.num")
    like_num = [like.text for like in like_list ]
    # 댓글 수 
    rep_list = driver.find_elements_by_css_selector("span.lo_txt")
    rep_num = [rep.text for rep in rep_list]

    #5.수집 데이터 dict로 구성 후 반환
    dict_sub ={}
    dict_sub["기사제목"] = title.text
    dict_sub["기사 입력 시간"]=date_time.text
    dict_sub["좋아요 수"] = like_num
    dict_sub["댓글 수"] = rep_num
    
    return dict_sub


In [44]:
url = "https://news.naver.com"
headers={"User-Agent":
             "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
result = requests.get(url,headers=headers)

bs_obj = BeautifulSoup(result.content,"html.parser")

#ul 태그 추출 : class:hdline_article_list
news_list = bs_obj.find("ul",{"class":"hdline_article_list"})
#모든 li 추출
lis = news_list.findAll("li")

#헤드라인뉴스 link 저장 - list
link_list=[]
for li in lis :
    a_tag = li.find("a")
    link_list.append("https://news.naver.com/"+a_tag["href"])
print(link_list)


driver = webdriver.Chrome("./driver/chromedriver")

import time

for number in range(len(link_list)) :
    res = get_news_sub_info(link_list[number])
    print(res)
    time.sleep(2)



['https://news.naver.com//main/read.naver?mode=LSD&mid=shm&sid1=101&oid=014&aid=0004688936', 'https://news.naver.com//main/read.naver?mode=LSD&mid=shm&sid1=102&oid=214&aid=0001140739', 'https://news.naver.com//main/read.naver?mode=LSD&mid=shm&sid1=102&oid=052&aid=0001624962', 'https://news.naver.com//main/read.naver?mode=LSD&mid=shm&sid1=101&oid=421&aid=0005530797', 'https://news.naver.com//main/read.naver?mode=LSD&mid=shm&sid1=102&oid=005&aid=0001463090']
{'기사제목': "17명 사상 광주 건물 붕괴, 무리한 해체+과도한 성토+안전 미비 '복합적 人災'", '기사 입력 시간': '2021.08.09. 오전 10:01', '좋아요 수': [], '댓글 수': ['']}
{'기사제목': '전두환, 항소심 법정 곧 출석…이 시각 광주', '기사 입력 시간': '2021.08.09. 오후 12:08', '좋아요 수': ['6'], '댓글 수': ['8']}
{'기사제목': "중고로 산 김치 냉장고 안에 '1억 1천만 원 돈다발'이...", '기사 입력 시간': '2021.08.09. 오후 2:51', '좋아요 수': ['23'], '댓글 수': ['18']}
{'기사제목': 'KCGI 손잡은 에디슨모터스 "쌍용차 3~5년내 흑자 자신…테슬라와 경쟁"', '기사 입력 시간': '2021.08.09. 오전 11:30', '좋아요 수': ['14'], '댓글 수': ['']}
{'기사제목': '‘생후 2주된 아들을…’ 학대·살해한 친부 징역 25년 선고', '기사 입력 시간': '2021.08.09. 오전 11:0